## TP parte A - Ruleta, Crossover Mutacion

### Librerias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Variables

In [2]:
PROB_CROSSOVER = 0.75
PROB_MUTACION = 0.05
CICLOS = 200
CANTIDAD_POBLACION = 10
COEF = 1073741823
ITERACIONES_IMPRIMIBLES = [20, 100, 200]

### Fitness y Objetivo

In [3]:
def convertirPoblacion(pob_ini, bin=True): # Pasar False para convertir a decimal
    pob_fin = []
    for numero in pob_ini:
        if bin:
            pob_fin.append(format(numero,'030b'))
        else:
            pob_fin.append(int(str(numero), 2))
    return pob_fin

In [4]:
def funcionObjetivo(poblacion_binarios):
    poblacion = convertirPoblacion(poblacion_binarios, False)
    fun_objetivo = []
    for cromosoma in poblacion:
        fun_objetivo.append((cromosoma/COEF)**2)
    return fun_objetivo

In [5]:
def funcionFitness(poblacion_binarios):
    fitness = []
    fun_objetivo = funcionObjetivo(poblacion_binarios)
    for objetivo in fun_objetivo:
        fitness.append((objetivo/sum(fun_objetivo)))
    return fitness

#### Poblacion

In [6]:
def poblacionInicial():
  rng = np.random.default_rng() # Random Number Generator
  poblacion_decimal = rng.integers(low=0, high=COEF, size=CANTIDAD_POBLACION)
  return poblacion_decimal

poblacion_decimal = poblacionInicial()
poblacion_decimal

array([174919638, 782122574, 590681511, 371771995, 255531878,  60695670,
       284247083, 468322121, 519401944, 684362234], dtype=int64)

#### Binarios

In [7]:
poblacion_binarios = convertirPoblacion(poblacionInicial(), True)
poblacion_binarios

['110100100001111011000011111001',
 '101100110101100010100011100101',
 '001101110100010000101000001011',
 '000000010011110011001011110100',
 '110100101111111011110011011110',
 '011101011010010011011100100001',
 '000110001111001110110111100100',
 '010010111001110100011111100011',
 '011000100000000001110000111011',
 '010000101001000101100100010011']

### Tabla de datos

In [8]:
def generarDataFrame(data):
    return pd.DataFrame(data)

datos  = {'Decimales': poblacion_decimal, 'Binarios': poblacion_binarios,
          'FuncionObjetivo':np.array(funcionObjetivo(poblacion_binarios)),
          'Fitness': np.array(funcionFitness(poblacion_binarios))
                  }

data_frame = generarDataFrame(datos)
data_frame

,Decimales,Binarios,FuncionObjetivo,Fitness
0,174919638,110100100001111011000011111001,0.673683,0.279245
1,782122574,101100110101100010100011100101,0.490800,0.203439
2,590681511,001101110100010000101000001011,0.046606,0.019318
3,371771995,000000010011110011001011110100,0.000023,0.000010
4,255531878,110100101111111011110011011110,0.679310,0.281578
5,60695670,011101011010010011011100100001,0.211183,0.087537
6,284247083,000110001111001110110111100100,0.009500,0.003938
7,468322121,010010111001110100011111100011,0.087241,0.036162
8,519401944,011000100000000001110000111011,0.146551,0.060746
9,684362234,010000101001000101100100010011,0.067616,0.028027


#### Estadisticas Marco de datos

In [9]:
def generarEstadisticas(dataFrame): 

    calcular_datos = lambda nombre,frame: ( frame[nombre].sum(), frame[nombre].mean(), frame[nombre].max(), frame[nombre].min() )


    fObjetivo_suma, fObjetivo_minimo, fObjetivo_promedio, fObjetivo_maximo = calcular_datos('FuncionObjetivo',dataFrame)
    fFitness_suma, fFitness_minimo, fFitness_promedio, fFitness_maximo = calcular_datos('Fitness',dataFrame)

    stats = {
        'Suma': [       fObjetivo_suma,     fFitness_suma ],
        'Promedio':[    fObjetivo_promedio, fFitness_promedio ],
        'Máximo': [     fObjetivo_maximo,   fFitness_maximo ], 
        'Mínimo': [     fObjetivo_minimo,   fFitness_minimo ]
        }
    
    dataFrame_stats = pd.DataFrame(stats, index = ['Función objetivo', 'Fitness'])

    return dataFrame_stats, stats

marcoDeDatos_stats, stats = generarEstadisticas(data_frame)
mc = marcoDeDatos_stats.loc['Fitness']
mc

Suma        1.000000
Promedio    0.100000
Máximo      0.281578
Mínimo      0.000010
Name: Fitness, dtype: float64

### Ruleta

In [10]:
def ruleta(pob_bin, cantidad_poblacion):
    # Asigna probabilidad basada en el fitness
    cromosomas = np.array(pob_bin)
    probabilidades = np.array(funcionFitness(pob_bin))
    np.random.seed()
    binarios_ruleta = np.random.choice(cromosomas, size=int(cantidad_poblacion), p=probabilidades)
    binarios_ruleta = binarios_ruleta.tolist()
    return binarios_ruleta
binarios_ruleta = ruleta(poblacion_binarios, 10)
binarios_ruleta

['010000101001000101100100010011',
 '110100100001111011000011111001',
 '110100101111111011110011011110',
 '110100100001111011000011111001',
 '110100100001111011000011111001',
 '101100110101100010100011100101',
 '101100110101100010100011100101',
 '011101011010010011011100100001',
 '101100110101100010100011100101',
 '101100110101100010100011100101']

### Crossover

In [11]:
def crossover(poblacion_binarios, prob_corssover):
    hijos_crossover = []
    pob_binarios = np.array(poblacion_binarios)
    cantidad_poblacion = len(pob_binarios)
    genes = len(pob_binarios[0])
    for i in range(0, cantidad_poblacion, 2):
        opciones = [True, False]
        np.random.seed()
        # Probabilidades de cada opción
        prob_cross = np.array([prob_corssover, (1-prob_corssover)])
        cross = np.random.choice(opciones, size=1, p=prob_cross)
        if cross[0]:
            posiciones = [x for x in range(0, genes)]
            probCorte = [(1/genes) for x in range(0, genes)]
            # Devuelve ndarray de 1 elemento
            corte = np.random.choice(posiciones, size=1, p=probCorte)
            posicionCorte = corte[0]
            primerTiraGenesPadre1 = pob_binarios[i][0:posicionCorte]
            segundaTiraGenesPadre1 = pob_binarios[i][posicionCorte:genes]
            primerTiraGenesPadre2 = pob_binarios[i+1][0:posicionCorte]
            segundaTiraGenesPadre2 = pob_binarios[i+1][posicionCorte:genes]
            hijos_crossover.append(primerTiraGenesPadre1 + segundaTiraGenesPadre2)  # Hijo 1
            hijos_crossover.append(primerTiraGenesPadre2 + segundaTiraGenesPadre1)  # Hijo 2
        else:
            hijos_crossover.append(pob_binarios[i])
            hijos_crossover.append(pob_binarios[i+1])
    return hijos_crossover
binarios_crossover = crossover(binarios_ruleta, PROB_CROSSOVER)
binarios_crossover

['010000101001000101100100010011',
 '110100100001111011000011111001',
 '110100101111111011000011111001',
 '110100100001111011110011011110',
 '110100100001111011000011111001',
 '101100110101100010100011100101',
 '101101011010010011011100100001',
 '011100110101100010100011100101',
 '101100110101100010100011100101',
 '101100110101100010100011100101']

### Mutacion

In [12]:
def mutacion(hijos_binarios, prob_mutacion):
        hijos_mutados = []
        genes = len(str(hijos_binarios[0]))
        for hijo in hijos_binarios:
            opciones = [True, False]
            np.random.seed()
            # Probabilidades de cada opción
            prob_mut = np.array([prob_mutacion, (1-prob_mutacion)])
            mut = np.random.choice(opciones, size=1, p=prob_mut)
            if mut[0]:
                posiciones = [x for x in range(0, genes)]
                probMutacion = [1/genes for x in range(0, genes)]
                # Devuelve ndarray de 1 elemento
                posicionMutacion = np.random.choice(posiciones, size=1, p=probMutacion)
                posicionMutacion = posicionMutacion[0]

                hijo[posicionMutacion] == '1' if hijo[posicionMutacion] == '0' else '0'

            hijos_mutados.append(hijo)
        return hijos_mutados
mutados = mutacion(binarios_crossover, PROB_MUTACION)
mutados

['010000101001000101100100010011',
 '110100100001111011000011111001',
 '110100101111111011000011111001',
 '110100100001111011110011011110',
 '110100100001111011000011111001',
 '101100110101100010100011100101',
 '101101011010010011011100100001',
 '011100110101100010100011100101',
 '101100110101100010100011100101',
 '101100110101100010100011100101']

### Torneo

In [13]:
def torneo(poblacion_binarios, cantidad_poblacion):
    binarios_torneo = []
    pob_binarios = np.array(poblacion_binarios)
    fitness = np.array(funcionFitness(pob_binarios))
    for i in range(0, cantidad_poblacion):
        posiblesCantidades = [x for x in range(1, (cantidad_poblacion+1))]
        np.random.seed()
        cantidadMiembrosTorneo = np.random.choice(posiblesCantidades, size=1)
        # Trabaja directamente con los fitness de los cromósomas
        miembrosTorneo = np.random.choice(fitness, size=cantidadMiembrosTorneo[0], replace=False)
        ganador = np.amax(miembrosTorneo)
        indiceGanador = np.where(fitness == ganador)  # retorna ndarray
        cromosomaGanador = pob_binarios[indiceGanador[0][0]]
        binarios_torneo.append(cromosomaGanador)
    return binarios_torneo
binarios_torneo = torneo(poblacion_binarios, 10)
binarios_torneo

['101100110101100010100011100101',
 '110100101111111011110011011110',
 '110100101111111011110011011110',
 '110100101111111011110011011110',
 '001101110100010000101000001011',
 '101100110101100010100011100101',
 '110100101111111011110011011110',
 '110100101111111011110011011110',
 '110100100001111011000011111001',
 '110100101111111011110011011110']

### Elitismo

In [14]:
def elitismo(poblacion_binarios, cantidad):
    fitness = funcionFitness(poblacion_binarios)
    elites = []
    for i in range(0, cantidad):
        ganador = max(fitness)
        indiceGanador = fitness.index(ganador)
        cromosomaGanador = poblacion_binarios[indiceGanador]
        elites.append(cromosomaGanador)
        fitness.remove(ganador)
    return elites
elites = elitismo(poblacion_binarios, 2)
elites

['110100101111111011110011011110', '110100100001111011000011111001']

## Iteracion

### Tablas

In [15]:
def hacerTabla(cantIteraciones, maximos, minimos, promedios):
    iteraciones = [x for x in range(1,cantIteraciones+1)]
    data = {'Iteracion': iteraciones, 'Maximo': maximos, 
            'Minimo':minimos, 'Promedio':promedios}
    data_frame = pd.DataFrame(data)
    return data_frame

### Graficos

In [16]:
def hacerGrafico(estadisticas, iteracion):
    estadisticas["Iteración"]=[x for x in range(1,iteracion+1)]
    plt.figure(figsize=(20,20))
    plt.plot(estadisticas["Iteración"], estadisticas["Máximo"], 'b.-', label="Máximo")
    plt.plot(estadisticas["Iteración"], estadisticas["Mínimo"], 'r.-', label="Mínimo")
    plt.plot(estadisticas["Iteración"], estadisticas["Promedio"], 'g.-', label="Promedio")
    plt.legend()
    plt.title("Gráfica iteración: " + str(iteracion))
    plt.xticks(estadisticas["Iteración"][::1])
    plt.yticks(np.arange(0, 1.01, step=0.025))
    plt.xlabel("Iteraciones")
    plt.ylabel("F. Objetivo")
    plt.show()

### Iteracion

In [17]:
def main(seleccion, ciclos, tieneElitismo = False, mutar = True):
    estadisticas = { "Máximo":[], "Mínimo":[], "Promedio":[]}
  
    
    poblacion_decimal = poblacionInicial()
    poblacion_binarios = convertirPoblacion(poblacion_decimal)
    
    for i in range(0, ciclos):
        
        datos  = {
            'Decimales':        poblacion_decimal, 
            'Binarios':         poblacion_binarios,
            'FuncionObjetivo':  np.array(funcionObjetivo(poblacion_binarios)),
            'Fitness':          np.array(funcionFitness(poblacion_binarios))
            }
        
        marcoDeDatos = generarDataFrame(datos)
        marcoDeDatos_stats, stats =  generarEstadisticas(marcoDeDatos)
        
        
        estadisticas["Máximo"].append(stats['Máximo'][0])
        estadisticas["Mínimo"].append(stats['Mínimo'][0])
        estadisticas["Promedio"].append(stats['Promedio'][0])
        
        
        if tieneElitismo:
            elites = elitismo(poblacion_binarios, 2)
            cromosomasPadres = seleccion(poblacion_binarios, 8)
        else:
            cromosomasPadres = seleccion(poblacion_binarios, 10)
        cromosomasHijos = crossover(cromosomasPadres, PROB_CROSSOVER)
        
        if mutar: 
            cromosomasHijosMutados = mutacion(cromosomasHijos, PROB_MUTACION)
            if tieneElitismo: 
                poblacion_binarios = cromosomasHijosMutados + elites
            else:
                poblacion_binarios = cromosomasHijosMutados
        else:
            if tieneElitismo: 
                poblacion_binarios = cromosomasHijos + elites
            else:
                poblacion_binarios = cromosomasHijos

        if ( i == ciclos-1 ):
            marcoDeDatos = generarDataFrame(poblacion_binarios)
            marcoDeDatos_stats, stats = generarEstadisticas(marcoDeDatos)
            tabla = hacerTabla(i+1, estadisticas["Máximo"], estadisticas["Mínimo"], estadisticas["Promedio"])
            display(tabla.style.hide(axis='index'))
            #display(marcoDeDatos_stats.style.set_table_attributes("style='display:inline'").set_caption('Tabla iteración: ' + str(i+1)))
            hacerGrafico(estadisticas, i+1)


debeMutar = True
conElitismo = True 

casos = { 
    "CASO A: Método selección ruleta": ruleta, 
    "CASO B: Método selección torneo": torneo, 
    "CASO C: Método selección Ruleta con elitismo": (ruleta, conElitismo, debeMutar), 
    "CASO C: Método selección torneo con elitismo": (torneo, conElitismo,debeMutar),

    
    "CASO A: Método selección ruleta sin mutacion": (ruleta, not conElitismo,not debeMutar), 
    "CASO B: Método selección torneo sin mutación": (torneo, not conElitismo,not debeMutar), 
    "CASO C: Método selección Ruleta con elitismo sin mutacion": (ruleta, conElitismo,not debeMutar), 
    "CASO C: Método selección torneo con elitismo sin mutacion": (torneo, conElitismo,not debeMutar) 
}  
 
 
for key, value in casos.items(): 
    print(key) 
    tipo_seleccion,conElitismo, conMutacion = (value[0],value[1],value[2]) if type(value) == tuple else (value,False,True) 
    for ciclo in ITERACIONES_IMPRIMIBLES: 
        main(tipo_seleccion, ciclo,conElitismo,conMutacion)



# print("CASO A: Método selección ruleta")
# main(ruleta)
# print("CASO B: Método selección torneo")
# main(torneo)
# print("CASO C: Método selección Ruleta con elitismo")
# main(ruleta, tieneElitismo = True)
# print("CASO C: Método selección torneo con elitismo")
# main(torneo, tieneElitismo = True)

# # Sin mutacion (Se ve como se estanca la función en maximos locales)
# print("-----------------Sin mutación-----------------")
# print("CASO A: Método selección ruleta sin mutación")
# main(ruleta, mutar = False)
# print("CASO B: Método selección torneo sin mutación")
# main(torneo, mutar = False)
# print("CASO C: Método selección Ruleta con elitismo sin mutación")
# main(ruleta, tieneElitismo = True, mutar = False)
# print("CASO C: Método selección torneo con elitismo sin mutación")
# main(torneo, tieneElitismo = True, mutar = False)

CASO A: Método selección ruleta


KeyError: 'FuncionObjetivo'